# Richards equation

Let $\Omega=(0,2)\times(0,3)$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$k$ the matrix permeability, we want to solve the following problem: find $({q}, h)$ such that
$$
\partial_t \theta (\psi) - \nabla \cdot (K \nabla h) = 0 \text{ in } \Omega \times (0,T)
$$
with boundary conditions:

$$
h(t,x,z)=
\left\{
\begin{array}{ll}
\begin{array}{l} 
1+2.2 \frac{t}{\Delta t_D}, \text{ on } \Gamma_{D_1}, t\leq\Delta t_D\\
3.2, \text{ on } \Gamma_{D_1}, t>\Delta t_D\\
1, \text{ on } \Gamma_{D_2}
\end{array}
\end{array}
\right., \qquad \nu \cdot K \nabla h = 0 \text{ on } \Gamma_N \qquad h(0,x,z) = 1 \text{ on } \Omega$$
and
$$
\Gamma_{D_1} = \left\{ (x,z) \in \partial \Omega \:|\: x \in [0,1] \wedge z=3  \right\},\\
\Gamma_{D_2} = \left\{ (x,z) \in \partial \Omega \:|\: x = 2 \wedge z \in [0,1]  \right\},\\
\Gamma_{D} = \Gamma_{D_1} \cup \Gamma_{D_2},\\
\Gamma_{N} = \partial \Omega \setminus \Gamma_D
$$

In [1]:
import shutil
import os

import numpy as np

import porepy as pp
import pygeon as pg

import time

import matplotlib.pyplot as plt

/opt/conda/lib/python3.11/site-packages/porepy/numerics/nonlinear/nonlinear_solvers.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import trange  # type: ignore


In [2]:
from richards.model_params import Model_Data
from richards.matrix_computer import Matrix_Computer

from richards.solver import Solver
from richards.solver_params import Solver_Data, Solver_Enum, Norm_Error

In [3]:
# Set the maximum number of iterations of the non-linear solver
K = 500

# L-scheme parameter
L = 3.501e-2

# Set the mesh refinment
N = 10

# Set the number of steps (excluding the initial condition)
num_steps = 9

# Simulation time length
T = num_steps/48

# Time switch conditions (for the boundary condition)
dt_D = 1/16

# Relative and absolute tolerances for the non-linear solver
abs_tol = 1e-5
rel_tol = 1e-5

# Domain tolerance
domain_tolerance = 1 / (10 * N)

# Output directory
output_directory = 'single_stage_primal'

In [4]:
# Time step
dt   = (T-0)/num_steps

In [5]:
# Van Genuchten model parameters ( relative permeability model )
model_data = Model_Data(theta_r=0.131, theta_s=0.396, alpha=0.423, n=2.06, K_s=4.96e-2, T=T, num_steps=num_steps)

In [6]:
# Prepare the domain and its mesh
subdomain = pp.StructuredTriangleGrid([2*N, 3*N], [2,3])
pg.convert_from_pp(subdomain)

# Convert it to a mixed-dimensional grid
mdg = pg.as_mdg(subdomain)

In [7]:
key = "flow"

In [8]:
# Initial pressure function
def initial_pressure_func(x): 
    return 1

In [9]:
# Fake loop to extract the grid and its data (i.e. conductivity tensor)
subdomain, data = mdg.subdomains(return_data=True)[0]

# Gamma_D1 and Gamma_D2 boundary faces
gamma_d1 = np.logical_and(subdomain.nodes[0, :] > 0-domain_tolerance, np.logical_and(subdomain.nodes[0, :] < 1+domain_tolerance, subdomain.nodes[1, :] > 3-domain_tolerance))
gamma_d2 = np.logical_and(subdomain.nodes[0, :] > 2-domain_tolerance, np.logical_and(subdomain.nodes[1, :] > 0-domain_tolerance, subdomain.nodes[1, :] < 1+domain_tolerance))

gamma_d  = np.logical_or(gamma_d1, gamma_d2)
bc_essential = gamma_d

# Add a lambda function that generates for each time instant the (discretized) natural boundary conditions for the problem
bc_value = lambda t: np.array(gamma_d2, dtype=float) + np.array(gamma_d1, dtype=float) * min(3.2, 1 + 2.2 * t / dt_D)

In [10]:
if os.path.exists(output_directory):
    shutil.rmtree(output_directory)

In [11]:
cp = Matrix_Computer(mdg)

solver_data = Solver_Data(mdg=mdg, initial_solution=cp.P1.interpolate(subdomain, initial_pressure_func), 
                          scheme=Solver_Enum.NEWTON, 
                          bc_essential=lambda t: bc_essential, bc_essential_value=bc_value,
                          eps_psi_rel=rel_tol, eps_psi_abs=abs_tol,
                          max_iterations_per_step=K, L_Scheme_value=L,
                          output_directory=output_directory, report_name='primal',
                          primal=True, integration_order=2, prepare_plots=False, 
                          step_output_allowed=True,
                          shape_x=2*N+1, shape_y=3*N+1, norm_error=Norm_Error.EUCLIDIAN)

In [12]:
solver = Solver(model_data=model_data, solver_data=solver_data)

In [13]:
start = time.time()
solver.solve()
end = time.time()

print('')
print(end - start)

Csv_Exporter: A file with name primal_NEWTON_richards_solver.csv is detected. I'll delete it
Time 0.02083


<lambdifygenerated-1>:2: RuntimeWarning: invalid value encountered in power
  return select([less(h, z),True], [0.265*(0.1699265174169*(-h + z)**2.06 + 1)**(-0.514563106796116) + 0.131,0.396], default=nan)
<lambdifygenerated-3>:2: RuntimeWarning: invalid value encountered in power
  return select([less(h, z),True], [0.00446702828985547*(1 - (1 - 1.0*(0.1699265174169*(-h + z)**2.06 + 1)**(-1.0))**0.514563106796116)**2*(-h + z)**1.06*(0.1699265174169*(-h + z)**2.06 + 1)**(-1.25728155339806) + 0.0178681131594219*(1 - (1 - 1.0*(0.1699265174169*(-h + z)**2.06 + 1)**(-1.0))**0.514563106796116)*(1 - 1.0*(0.1699265174169*(-h + z)**2.06 + 1)**(-1.0))**(-0.485436893203884)*(-h + z)**1.06*(0.1699265174169*(-h + z)**2.06 + 1)**(-2.25728155339806),0], default=nan)
<lambdifygenerated-4>:2: RuntimeWarning: invalid value encountered in scalar power
  return select([less(h, z),True], [0.0477323587424072*(-h + z)**1.06*(0.1699265174169*(-h + z)**2.06 + 1)**(-1.51456310679612),0], default=nan)
<lambdifyg

Iteration #0001, relative norm of the error:    0.095819356, norm of the error:    2.444802280
Iteration #0002, relative norm of the error:    0.012325289, norm of the error:    0.320232576
Iteration #0003, relative norm of the error:    0.000456582, norm of the error:    0.011887428
Iteration #0004, relative norm of the error:    0.000003171, norm of the error:    0.000082560

Time 0.04167
Iteration #0001, relative norm of the error:    0.110289686, norm of the error:    2.871330258
Iteration #0002, relative norm of the error:    0.018469827, norm of the error:    0.498737241
Iteration #0003, relative norm of the error:    0.001015298, norm of the error:    0.027555348
Iteration #0004, relative norm of the error:    0.000022434, norm of the error:    0.000608744
Iteration #0005, relative norm of the error:    0.000000572, norm of the error:    0.000015515

Time 0.0625
Iteration #0001, relative norm of the error:    0.147315619, norm of the error:    3.997407533
Iteration #0002, relati